# Lightning AI SDXL LoRA Trainer

Este cuaderno prepara un entorno dentro de Lightning AI (ruta raíz ``/teamspace/studios/this_studio``)
y ejecuta el entrenador de LoRA para Stable Diffusion XL incluido en este fork.

Los pasos principales son:

1. Crear/usar un entorno gestionado por [uv](https://github.com/astral-sh/uv).
2. Instalar las dependencias necesarias.
3. Definir la configuración de entrenamiento (epochs, repeats, tasas de aprendizaje, etc.).
4. Ejecutar el entrenamiento calculando automáticamente los *steps* mediante la fórmula requerida.
5. Seleccionar optimizadores personalizados (AdamW8bit, Prodigy, DAdapt*, Lion, etc.) y schedulers (`cosine`, `rex`, `cosine_with_restarts`, entre otros) incluidos en este fork.


In [ ]:
from pathlib import Path
import os
import subprocess

LIGHTNING_ROOT = Path("/teamspace/studios/this_studio")
UV_BIN = Path.home() / ".local" / "bin" / "uv"

if not UV_BIN.exists():
    print("Instalando uv…")
    subprocess.run(
        ["/bin/bash", "-lc", "curl -LsSf https://astral.sh/uv/install.sh | sh"],
        check=True,
    )
else:
    print("uv ya está instalado")

os.environ["PATH"] = f"{UV_BIN.parent}:{os.environ['PATH']}"
os.environ.setdefault("UV_PROJECT_ENVIRONMENT", str(LIGHTNING_ROOT / ".venv"))
print("Entorno de uv:", os.environ["UV_PROJECT_ENVIRONMENT"])

In [ ]:
import subprocess

required_packages = [
    "bitsandbytes",
    "prodigyopt",
    "lion-pytorch",
    "dadaptation",
    "pytorch-optimizer==3.1.2",
    "torch>=2.1",
    "torchvision",
    "accelerate>=0.23",
    "diffusers[torch]>=0.24",
    "transformers>=4.35",
    "peft>=0.7",
    "safetensors",
]

print("Instalando dependencias con uv…")
subprocess.run(["uv", "pip", "install", "--upgrade", *required_packages], check=True)


In [ ]:
from pathlib import Path
from lightning_ai.sdxl_lora_trainer import TrainingConfig

base_config = TrainingConfig(
    dataset_metadata=Path("datasets/mi_dataset/metadata.jsonl"),
    images_root=Path("datasets/mi_dataset/images"),
    output_dir=Path("outputs/sdxl_lora_demo"),
    num_epochs=2,
    batch_size=1,
    gradient_accumulation=1,
    num_repeats=2,
    resolution=1024,
    network_rank=64,
    network_alpha=128,
    unet_lr=1e-4,
    text_encoder_lr=5e-6,
    optimizer_type="adamw",
    weight_decay=1e-2,
    optimizer_beta1=0.9,
    optimizer_beta2=0.999,
    optimizer_eps=1e-8,
    optimizer_momentum=0.9,
    scheduler_type="cosine",
    lr_warmup_steps=None,
    scheduler_first_cycle_steps=None,
    scheduler_cycle_multiplier=1.0,
    scheduler_gamma=1.0,
    scheduler_min_lr=1e-6,
    scheduler_d=0.9,
    scheduler_power=1.0,
    train_text_encoders=True,
    mixed_precision="fp16",
)

config = base_config.normalised_paths()
print("Configuración resuelta:")
for key, value in config.__dict__.items():
    print(f"  {key}: {value}")

In [ ]:
import json
from lightning_ai.sdxl_lora_trainer import calculate_total_steps

num_images = 0
with config.dataset_metadata.open("r", encoding="utf-8") as handle:
    for line in handle:
        if line.strip():
            num_images += 1

steps = calculate_total_steps(
    num_images=num_images,
    num_repeats=config.num_repeats,
    num_epochs=config.num_epochs,
    batch_size=config.batch_size,
)
print(json.dumps(
    {
        "num_images": num_images,
        "num_repeats": config.num_repeats,
        "num_epochs": config.num_epochs,
        "batch_size": config.batch_size,
        "calculated_steps": steps,
    },
    indent=2,
))

In [ ]:
from lightning_ai.sdxl_lora_trainer import train

print("Iniciando entrenamiento…")
train(config)